In [1]:
from mpi4py import MPI
from petsc4py import PETSc

from tqdm.notebook import tqdm_notebook

import numpy as np

import ufl

from dolfinx import fem, io, mesh, plot, la
from dolfinx.fem.petsc import assemble_matrix, create_vector, apply_lifting, NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from basix.ufl import element, mixed_element
from dolfinx.io import XDMFFile

import pyvista

In [2]:
# create mesh for calculations to take place in
msh = mesh.create_unit_square(MPI.COMM_WORLD, 16,16)

# define elements to be used for the velocity and the distribution
s_p1 = element("P", msh.basix_cell(), 1)

# define time step variables 
T = 5        # final time
num_steps = 500   # number of time steps
dt = T / num_steps # time step size

In [3]:
def initial_condition(x, a=10):
    return np.exp(-a * ((x[0]-.5)**2 + (x[1]-.5)**2))

In [4]:
v_p2 = element("P", msh.basix_cell(), 2, shape=(msh.geometry.dim,))
V = fem.FunctionSpace(msh, v_p2)
def flow(x):
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1])))

# very basic flow variable
u_n = fem.Function(V)
u_n.interpolate(flow)

In [5]:
bcs = []

In [6]:
S = fem.FunctionSpace(msh, s_p1)

s_ = fem.Function(S)
s_n = fem.Function(S)

s_.interpolate(initial_condition)
s_n.interpolate(initial_condition)

s = ufl.TrialFunction(S)
z = ufl.TestFunction(S)

In [7]:
eps = fem.Constant(msh, PETSc.ScalarType(0.1)) # diffusion coefficent
rho = fem.Constant(msh, PETSc.ScalarType(0))
k = fem.Constant(msh, dt)
f = fem.Constant(msh, PETSc.ScalarType(0))

In [8]:
F = 0

F += ((s_ - s_n) / k) * z * ufl.dx # advective terms
F +=  ufl.dot( u_n, ufl.grad(s_)) * z *ufl.dx # advective terms
F += eps * ufl.dot( ufl.grad( s_ ), ufl.grad( z ) ) * ufl.dx # diffusive term
F -=  f * z  * ufl.dx


prob = NonlinearProblem(F, s_, bcs=bcs)

solver = NewtonSolver(MPI.COMM_WORLD, prob)


# a = fem.form(ufl.lhs(F))
# L = fem.form(ufl.rhs(F))

In [9]:
# A = assemble_matrix(a, bcs=bcs)
# A.assemble()
# b = create_vector(L)

# solver = PETSc.KSP().create(msh.comm)
# solver.setOperators(A)
# solver.setType(PETSc.KSP.Type.PREONLY)
# solver.getPC().setType(PETSc.PC.Type.LU)

In [10]:
outputfile = io.XDMFFile( msh.comm, "Reportfigures/Diffusion.xdmf", "w")

outputfile.write_mesh( msh )

In [11]:
t = 0
outputfile.write_function( s_, t )

for i in tqdm_notebook( range( num_steps )):
    t += dt
    
    # with b.localForm() as loc_b:
    #     loc_b.set(0)
        
    # fem.petsc.assemble_vector(b, L)
    # apply_lifting(b, [a], [bcs])
    # b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    # fem.set_bc(b, bcs)
        
    # solver.solve(b, s_.vector)
    solver.solve(s_)
    
    s_.x.scatter_forward() # correct for inacuracies 

    s_n.x.array[:] = s_.x.array

    outputfile.write_function( s_, t )
outputfile.close()

  0%|          | 0/500 [00:00<?, ?it/s]